# TBL to STAR Conversion Utilities

This notebook demonstrates the use of conversion utilities from `utils.tbl2star` module:
- Convert Dynamo .tbl/.vll files to RELION .star format
- Convert RELION .star files to Dynamo .tbl format
- Add rlnTomoName column to .star files

In [ ]:
import sys
import os

# Add project root to Python path
notebook_dir = os.path.dirname(os.path.dirname(os.path.abspath('')))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

In [ ]:
from utils.tbl2star import tbl_to_star, star_to_tbl, add_rln_tomo_name_to_star
import starfile

## Configuration

Set your input/output file paths and parameters here.

In [ ]:
# Input files (Dynamo format)
input_tbl = "path/to/input.tbl"
input_vll = "path/to/input.vll"

# Output files
output_star = "path/to/output.star"
output_tbl = "path/to/output.tbl"
output_star_with_tomo_name = "path/to/output_tomo_name.star"

# Conversion parameters
pixel_size = 6.72  # Angstrom
tomogram_size = (999, 999, 499)  # (x, y, z) in pixels

## 1. Convert TBL to STAR

Convert Dynamo .tbl and .vll files to RELION .star format.

In [ ]:
tbl_to_star(
    tbl_path=input_tbl,
    vll_path=input_vll,
    output_file=output_star,
    pixel_size=pixel_size,
    tomogram_size=tomogram_size,
    output_centered=True
)

print(f"Conversion complete. Output written to: {output_star}")

In [ ]:
# Verify the output star file
df_star = starfile.read(output_star)
print(f"Number of particles: {len(df_star)}")
print(f"Columns: {list(df_star.columns)}")
print(f"\nFirst few rows:")
df_star.head()

## 2. Convert STAR to TBL

Convert RELION .star file back to Dynamo .tbl format.

In [ ]:

df_tbl = star_to_tbl(
    star_path=output_star,
    pixel_size=pixel_size,
    tomogram_size=tomogram_size,
    output_file=output_tbl
)

print(f"Conversion complete. Output written to: {output_tbl}")
print(f"\nDataFrame shape: {df_tbl.shape}")
df_tbl.head()

## 3. Add rlnTomoName Column to STAR

Add rlnTomoName column to a RELION .star file by mapping unique micrograph names to sequential integer IDs.

In [ ]:
output_path = add_rln_tomo_name_to_star(
    star_path=output_star,
    output_file=output_star_with_tomo_name,
    use_micrograph_name=True
)

print(f"rlnTomoName column added. Output written to: {output_path}")

In [ ]:
# Verify the rlnTomoName column was added
df_star_with_tomo = starfile.read(output_path)
print(f"Columns: {list(df_star_with_tomo.columns)}")
print(f"\nUnique tomogram names and their IDs:")
if 'rlnTomoName' in df_star_with_tomo.columns:
    unique_tomos = df_star_with_tomo[['rlnMicrographName', 'rlnTomoName']].drop_duplicates()
    print(unique_tomos.head(10))
    print(f"\nTotal unique tomograms: {df_star_with_tomo['rlnTomoName'].nunique()}")

## Complete Workflow Example

Example of a complete conversion workflow: TBL → STAR → STAR with rlnTomoName

In [ ]:
# Example: Complete workflow
# Step 1: TBL to STAR
tbl_to_star(
    tbl_path=input_tbl,
    vll_path=input_vll,
    output_file=output_star,
    pixel_size=pixel_size,
    tomogram_size=tomogram_size,
    output_centered=True
)

# Step 2: Add rlnTomoName
final_output = add_rln_tomo_name_to_star(
    star_path=output_star,
    use_micrograph_name=True
)

print(f"Complete workflow finished.")
print(f"Intermediate STAR file: {output_star}")
print(f"Final STAR file with rlnTomoName: {final_output}")